In [0]:
!apt-get update && apt-get install -y imagemagick

In [5]:
from getpass import getpass
import boto3
access_id = getpass('access id: ')
secret = getpass('secret id: ')

access id: ··········
secret id: ··········


In [0]:
# copied from https://github.com/CMU-Perceptual-Computing-Lab/openpose/issues/949
! wget -c "https://github.com/Kitware/CMake/releases/download/v3.13.4/cmake-3.13.4.tar.gz"
! tar xf cmake-3.13.4.tar.gz
! cd cmake-3.13.4 && ./configure && make && sudo make install

# ライブラリのインストール

# Basic
! sudo apt-get --assume-yes update
! sudo apt-get --assume-yes install build-essential
# OpenCV
! sudo apt-get --assume-yes install libopencv-dev
# General dependencies
! sudo apt-get --assume-yes install libatlas-base-dev libprotobuf-dev libleveldb-dev libsnappy-dev libhdf5-serial-dev protobuf-compiler
! sudo apt-get --assume-yes install --no-install-recommends libboost-all-dev
# Remaining dependencies, 14.04
! sudo apt-get --assume-yes install libgflags-dev libgoogle-glog-dev liblmdb-dev
# Python2 libs
! sudo apt-get --assume-yes install python-setuptools python-dev build-essential
! sudo easy_install pip
! sudo -H pip install --upgrade numpy protobuf opencv-python
# Python3 libs
! sudo apt-get --assume-yes install python3-setuptools python3-dev build-essential
! sudo apt-get --assume-yes install python3-pip
! sudo -H pip3 install --upgrade numpy protobuf opencv-python
# OpenCV 2.4 -> Added as option
# # sudo apt-get --assume-yes install libopencv-dev
# OpenCL Generic
! sudo apt-get --assume-yes install opencl-headers ocl-icd-opencl-dev
! sudo apt-get --assume-yes install libviennacl-dev

#  Openpose の clone
#! git clone  --depth 1 -b "$ver_openpose" https://github.com/CMU-Perceptual-Computing-Lab/openpose.git 
! git clone  --depth 1 https://github.com/CMU-Perceptual-Computing-Lab/openpose.git     

#  Openpose の モデルデータDL
! cd openpose/models && ./getModels.sh

# Openpose の ビルド
! sed -i 's/execute_process(COMMAND git checkout master WORKING_DIRECTORY ${CMAKE_SOURCE_DIR}\/3rdparty\/caffe)/execute_process(COMMAND git checkout f019d0dfe86f49d1140961f8c7dec22130c83154 WORKING_DIRECTORY ${CMAKE_SOURCE_DIR}\/3rdparty\/caffe)/g' openpose/CMakeLists.txt
! cd openpose && rm -r build || true && mkdir build && cd build && cmake .. && make -j`nproc`



In [3]:
!git clone https://github.com/affinelayer/pix2pix-tensorflow.git
!cd pix2pix-tensorflow

Cloning into 'pix2pix-tensorflow'...
remote: Enumerating objects: 261, done.
remote: Total 261 (delta 0), reused 0 (delta 0), pack-reused 261
Receiving objects: 100% (261/261), 13.33 MiB | 13.09 MiB/s, done.
Resolving deltas: 100% (103/103), done.


In [0]:
s3r = boto3.resource('s3', aws_access_key_id=access_id,
    aws_secret_access_key=secret)
bucket = s3r.Bucket('insight-ai-project-en-asl')
bucket.download_file('pose2vid/data/ASLLVD-ASL_2007_05_24-scene3-camera1.mov', 'scene3-camera1.mov')

In [30]:
# 922.669 seconds to run on 21 second video
! cd openpose && ./build/examples/openpose/openpose.bin -disable-blending=True --face --hand --video ../scene3-camera1.mov --display 0  --write_video ../output/pose.mov

Starting OpenPose demo...
Configuring OpenPose...
Starting thread(s)...
Auto-detecting all available GPUs... Detected 1 GPU(s), using 1 of them starting at GPU 0.
OpenCV: FFMPEG: tag 0x47504a4d/'MJPG' is not supported with codec id 8 and format 'mov / QuickTime / MOV'
OpenCV: FFMPEG: fallback to use tag 0x6765706a/'jpeg'
OpenPose demo successfully finished. Total time: 146.271184 seconds.




Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
Instructions for updating:
This op will be removed after the deprecation date. Please switch to tf.sets.difference().
Traceback (most recent call last):
  File "tools/process.py", line 306, in <module>
    main()
  File "tools/process.py", line 242, in main
    for src_path in im.find(a.input_dir):
  File "/content/pix2pix-tensorflow/tools/tfimage.py", line 115, in find
    for filename in os.listdir(d):
FileNotFoundError: [Errno 2] No such file or directory: 'images/pose/'


In [32]:
from moviepy.editor import *
clip = VideoFileClip("output/pose.mov")
clip.ipython_display(5000)

100%|█████████▉| 564/565 [00:01<00:00, 291.47it/s]


In [0]:
!mkdir -p images/person/
!mkdir -p images/pose/
!mkdir -p images/combined/

In [34]:
!ffmpeg -i scene3-camera1.mov images/person/train-video-%05d.jpg -hide_banner

Input #0, mov,mp4,m4a,3gp,3g2,mj2, from 'scene3-camera1.mov':
  Metadata:
    creation_time   : 2011-08-18T07:28:10.000000Z
  Duration: 00:00:09.40, start: 0.000000, bitrate: 26949 kb/s
    Stream #0:0(eng): Video: mpeg4 (Simple Profile) (mp4v / 0x7634706D), yuv420p, 640x480 [SAR 1:1 DAR 4:3], 26944 kb/s, 60 fps, 60 tbr, 120k tbn, 25 tbc (default)
    Metadata:
      creation_time   : 2011-08-18T07:28:10.000000Z
      handler_name    : 3ivx MPEG-4 Alias Data Handler
      encoder         : 3ivx MPEG-4 5.0.2
Stream mapping:
  Stream #0:0 -> #0:0 (mpeg4 (native) -> mjpeg (native))
Press [q] to stop, [?] for help
[swscaler @ 0x5641b42ce000] deprecated pixel format used, make sure you did set range correctly
Output #0, image2, to 'images/person/train-video-%05d.jpg':
  Metadata:
    encoder         : Lavf57.83.100
    Stream #0:0(eng): Video: mjpeg, yuvj420p(pc), 640x480 [SAR 1:1 DAR 4:3], q=2-31, 200 kb/s, 60 fps, 60 tbn, 60 tbc (default)
    Metadata:
      creation_time   : 2011-08-18T0

In [35]:
!ffmpeg -i output/pose.mov images/pose/train-video-%05d.jpg -hide_banner

Input #0, mov,mp4,m4a,3gp,3g2,mj2, from 'output/pose.mov':
  Metadata:
    major_brand     : qt  
    minor_version   : 512
    compatible_brands: qt  
    encoder         : Lavf57.83.100
  Duration: 00:00:09.40, start: 0.000000, bitrate: 7391 kb/s
    Stream #0:0(eng): Video: mjpeg (jpeg / 0x6765706A), yuvj420p(pc, bt470bg/unknown/unknown), 640x480, 7389 kb/s, 60 fps, 60 tbr, 15360 tbn, 15360 tbc (default)
    Metadata:
      handler_name    : DataHandler
Stream mapping:
  Stream #0:0 -> #0:0 (mjpeg (native) -> mjpeg (native))
Press [q] to stop, [?] for help
Output #0, image2, to 'images/pose/train-video-%05d.jpg':
  Metadata:
    major_brand     : qt  
    minor_version   : 512
    compatible_brands: qt  
    encoder         : Lavf57.83.100
    Stream #0:0(eng): Video: mjpeg, yuvj420p(pc), 640x480, q=2-31, 200 kb/s, 60 fps, 60 tbn, 60 tbc (default)
    Metadata:
      handler_name    : DataHandler
      encoder         : Lavc57.107.100 mjpeg
    Side data:
      cpb: bitrate max/min/

In [0]:
!mogrify -format png images/person/*.jpg
!mogrify -format png images/pose/*.jpg
!rm -rf images/pose/*.jpg
!rm -rf images/person/*.jpg

In [0]:
!cd pix2pix-tensorflow && python tools/process.py \
  --input_dir ../images/pose/ \
  --b_dir ../images/person/ \
  --operation combine \
  --output_dir ../images/combined/

In [0]:
!cd pix2pix-tensorflow && python tools/split.py \
  --dir ../images/combined/

In [0]:
!tar -czvf train-images.tar.gz images/

In [0]:
bucket.upload_file('train-images.tar.gz', 'pose2vid/data/train-images.tar.gz')

In [0]:
!cd pix2pix-tensorflow && python pix2pix.py \
  --mode train \
  --output_dir ../pose2vid_train/ \
  --max_epochs 200 \
  --input_dir ../images/combined/train/ \
  --which_direction AtoB



aspect_ratio = 1.0
batch_size = 1
beta1 = 0.5
checkpoint = None
display_freq = 0
flip = True
gan_weight = 1.0
input_dir = ../images/combined/train/
l1_weight = 100.0
lab_colorization = False
lr = 0.0002
max_epochs = 200
max_steps = None
mode = train
ndf = 64
ngf = 64
output_dir = ../pose2vid_train/
output_filetype = png
progress_freq = 50
save_freq = 5000
scale_size = 286
seed = 436300208
separable_conv = False
summary_freq = 100
trace_freq = 0
which_direction = AtoB
Instructions for updating:
Queue-based input pipelines have been replaced by `tf.data`. Use `tf.data.Dataset.from_tensor_slices(string_tensor).shuffle(tf.shape(input_tensor, out_type=tf.int64)[0]).repeat(num_epochs)`. If `shuffle=False`, omit the `.shuffle(...)`.
Instructions for updating:
Queue-based input pipelines have been replaced by `tf.data`. Use `tf.data.Dataset.from_tensor_slices(input_tensor).shuffle(tf.shape(input_tensor, out_type=tf.int64)[0]).repeat(num_epochs)`. If `shuffle=False`, omit the `.shuffle(...)`.


In [0]:
!cd pix2pix-tensorflow && python pix2pix.py \
  --mode train \
  --output_dir ../pose2vid_train/ \
  --max_epochs 200 \
  --input_dir ../images/combined/train/ \
  --which_direction AtoB
# test the model
!cd pix2pix-tensorflow && python pix2pix.py \
  --mode test \
  --output_dir ../pose2vid_test/ \
  --input_dir ../images/combined/val/ \
  --checkpoint ../pose2vid_train/